In [2]:
import pandas as pd
from pyproj import Proj, transform
import fiona
from fiona.crs import from_epsg
import logging
import sys


# outProj = Proj(init='epsg:3857')
# inProj = Proj(init='epsg:4326') # It is equal to WSG

In [3]:
shape = fiona.open("Mapas_sim/base/conservacao.shp")
print (shape.schema)
# Chico Mendes é o id 73

{'properties': OrderedDict([('id', 'float:19'), ('Unidade_de', 'str:254'), ('Categoria', 'str:254'), ('Ano_de_Cri', 'int:9'), ('Zoneamento', 'str:254'), ('Área_hecta', 'float:33.15'), ('Jurisdição', 'str:254'), ('Plano_de_M', 'str:254')]), 'geometry': 'Polygon'}


In [4]:
shape[0]

{'type': 'Feature',
 'id': '0',
 'properties': OrderedDict([('id', 2.0),
              ('Unidade_de', 'PARQUE NACIONAL DA TIJUCA'),
              ('Categoria', 'PROTEÇÃO INTEGRAL'),
              ('Ano_de_Cri', 1961),
              ('Zoneamento', None),
              ('Área_hecta', 3958.48),
              ('Jurisdição', 'FEDERAL'),
              ('Plano_de_M', None)]),
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[(-43.28014978099997, -22.983690089999996),
     (-43.28010428499994, -22.98370955499998),
     (-43.28006023399997, -22.983730742),
     (-43.280008008999964, -22.983756447999966),
     (-43.27995862599994, -22.983769089999967),
     (-43.27990085999994, -22.98377595899997),
     (-43.27985110399993, -22.983777868999947),
     (-43.27978691199997, -22.983778799999982),
     (-43.27973615399997, -22.983781500999953),
     (-43.279685396999994, -22.983791698999955),
     (-43.279634894999965, -22.98380622799994),
     (-43.27956632599995, -22.98382164099996),
     (

In [3]:
# Transformar escala lag, long em Metros no globo achatado

# salvando o esquema e pegando a projecao de entrada
sink_schema = shape.schema.copy()
p_in = Proj(shape.crs)

# definindo um novo shape de saida, no mesmo local do shape atual com a projeção mudada
with fiona.open(
        'area_chico_mendes.shp', 'w',
        crs=from_epsg(3857),
        driver=shape.driver,
        schema=sink_schema,
        ) as sink:

    p_out = Proj(sink.crs)

    # fazendo apenas para o Chico Mendes (esse arquivo tem Polygon e MultiPolygon)
    # nesse caso, se fizesse para o arquivo todo, nao seria contemplado alguns shapes
    f = shape[73]

    try:
        # verifica se eh da estrutura poligono
        assert f['geometry']['type'] == "Polygon"
        new_coords = []
        # para todas as coordenadas do poligono
        for ring in f['geometry']['coordinates']:
            x2, y2 = transform(p_in, p_out, *zip(*ring)) #transforma
            new_coords.append(zip(x2, y2)) # salva a nova lista de coordenadas
        f['geometry']['coordinates'] = new_coords
        sink.write(f)

    except Exception:
        # Writing uncleanable features to a different shapefile
        # is another option.
        logging.exception("Error transforming feature %s:", f['id'])

/opt/anaconda/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/opt/anaconda/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [4]:
for i in range(0, len(shape)):
    if shape[i]['properties']['Unidade_de'] == 'PARQUE NATURAL MUNICIPAL CHICO MENDES':
        chico = shape[i]

In [5]:
print(chico['geometry']['coordinates'])

[[(-43.46900242299945, -23.02168791899976), (-43.46900242299945, -23.02168791899976), (-43.46868186399948, -23.021686624999813), (-43.46840067299947, -23.021688170999763), (-43.468267365999466, -23.02169125999977), (-43.46813439599948, -23.021700530999794), (-43.46800209399948, -23.021715959999764), (-43.46787079399946, -23.021737508999752), (-43.46774082699944, -23.021765122999785), (-43.467612517999456, -23.021798733999784), (-43.46748619099947, -23.0218382559998), (-43.46736216399946, -23.02188358899978), (-43.46724074699945, -23.021934621999776), (-43.46712224799944, -23.02199122399979), (-43.467006961999466, -23.02205325399979), (-43.46689518199948, -23.022120554999788), (-43.466815307999475, -23.022174851999804), (-43.46673930899943, -23.02223372299978), (-43.46666748999946, -23.022296928999744), (-43.466600137999436, -23.022364216999783), (-43.46653752399943, -23.022435317999786), (-43.46647989999948, -23.022509945999776), (-43.46642749699948, -23.022587800999776), (-43.46638052

In [6]:
shape2 = fiona.open("area_chico_mendes.shp")
print (shape2.schema)

{'properties': OrderedDict([('id', 'float:19'), ('Unidade_de', 'str:254'), ('Categoria', 'str:254'), ('Ano_de_Cri', 'int:9'), ('Zoneamento', 'str:254'), ('Área_hecta', 'float:33.15'), ('Jurisdição', 'str:254'), ('Plano_de_M', 'str:254')]), 'geometry': 'Polygon'}


In [7]:
print(shape2[0]['geometry'])

{'type': 'Polygon', 'coordinates': [[(-4838947.215019874, -2634641.6353100897), (-4838947.215019874, -2634641.6353100897), (-4838911.530555228, -2634641.478797528), (-4838880.228516291, -2634641.665790127), (-4838865.3888489315, -2634642.0394124887), (-4838850.586696242, -2634643.1607634327), (-4838835.858904972, -2634645.0269402442), (-4838821.242655828, -2634647.6333469907), (-4838806.774795566, -2634650.973331637), (-4838792.491503024, -2634655.038669846), (-4838778.428845712, -2634659.8189602373), (-4838764.622223226, -2634665.3021081625), (-4838751.106144611, -2634671.474688582), (-4838737.914896271, -2634678.320857462), (-4838725.081317457, -2634685.823561296), (-4838712.638024779, -2634693.9638113813), (-4838703.74649177, -2634700.5311931195), (-4838695.286321784, -2634707.6518165837), (-4838687.291467278, -2634715.2967747194), (-4838679.793876932, -2634723.435467131), (-4838672.823718335, -2634732.0353581333), (-4838666.409044003, -2634741.061855826), (-4838660.575568726, -2634

In [8]:
sink_schema = shape2.schema.copy()
p_in = Proj(shape2.crs)

with fiona.open(
        'area_chico_mendes2.shp', 'w',
        crs=from_epsg(4326),
        driver=shape.driver,
        schema=sink_schema,
        ) as sink:

    p_out = Proj(sink.crs)

    f = shape2[0]

    try:
        assert f['geometry']['type'] == "Polygon"
        new_coords = []
        for ring in f['geometry']['coordinates']:
            x2, y2 = transform(p_in, p_out, *zip(*ring))
            new_coords.append(zip(x2, y2))
        f['geometry']['coordinates'] = new_coords
        sink.write(f)

    except Exception:
        # Writing uncleanable features to a different shapefile
        # is another option.
        logging.exception("Error transforming feature %s:", f['id'])

/opt/anaconda/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/opt/anaconda/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [9]:
shape3 = fiona.open("area_chico_mendes2.shp")
print (shape.schema)

{'properties': OrderedDict([('id', 'float:19'), ('Unidade_de', 'str:254'), ('Categoria', 'str:254'), ('Ano_de_Cri', 'int:9'), ('Zoneamento', 'str:254'), ('Área_hecta', 'float:33.15'), ('Jurisdição', 'str:254'), ('Plano_de_M', 'str:254')]), 'geometry': 'Polygon'}


In [10]:
shape3[0]

{'type': 'Feature',
 'id': '0',
 'properties': OrderedDict([('id', 897.0),
              ('Unidade_de', 'PARQUE NATURAL MUNICIPAL CHICO MENDES'),
              ('Categoria', 'PROTEÇÃO INTEGRAL'),
              ('Ano_de_Cri', None),
              ('Zoneamento', None),
              ('Área_hecta', 40.65),
              ('Jurisdição', 'MUNICIPAL'),
              ('Plano_de_M', None)]),
 'geometry': {'type': 'Polygon',
  'coordinates': [[(-43.46900242299945, -23.021687918999756),
    (-43.46900242299945, -23.021687918999756),
    (-43.46868186399947, -23.021686624999813),
    (-43.468400672999465, -23.021688170999763),
    (-43.468267365999466, -23.02169125999978),
    (-43.46813439599948, -23.0217005309998),
    (-43.46800209399948, -23.021715959999764),
    (-43.46787079399945, -23.021737508999752),
    (-43.46774082699944, -23.021765122999785),
    (-43.46761251799945, -23.021798733999788),
    (-43.46748619099947, -23.021838255999803),
    (-43.46736216399946, -23.021883588999785),
   